Dynamic Pricing Notebook for Urban Parking
Summer Analytics 2025 | CnA × Pathway

In [ ]:
# =======================
# Dynamic Pricing Notebook for Urban Parking
# Summer Analytics 2025 | CnA × Pathway
# =======================

# === Setup ===
!pip install -q bokeh pathway

import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import gridplot
from bokeh.io import push_notebook
import pathway as pw

output_notebook()

# === Load Dataset ===
df = pd.read_csv('/content/dataset.csv')  # Upload your CSV to Colab

# Preview data
df.head()


1. Data Exploration


In [ ]:
print("Columns:", df.columns.tolist())
print(df.dtypes)
print("Unique Locations:", df['ParkingLotID'].nunique())  # assuming a column like this
print("Time Range:", df['Time'].min(), "to", df['Time'].max())


2. Model 1: Baseline Linear Pricing

In [ ]:
# Set base price
BASE_PRICE = 10.0
ALPHA = 5.0  # Tuning parameter

def baseline_pricing(occupancy, capacity, prev_price):
    return prev_price + ALPHA * (occupancy / capacity)

df['Price_Model1'] = BASE_PRICE

# Apply model
for i in range(1, len(df)):
    prev_price = df.loc[i-1, 'Price_Model1']
    occupancy = df.loc[i, 'Occupancy']
    capacity = df.loc[i, 'Capacity']
    df.loc[i, 'Price_Model1'] = baseline_pricing(occupancy, capacity, prev_price)

df[['Time', 'Price_Model1']].head()


3. Visualization with Bokeh (Model 1)

In [ ]:
p = figure(title="Model 1 - Price over Time", x_axis_label='Time', x_axis_type='datetime', y_axis_label='Price')
df['Time'] = pd.to_datetime(df['Time'])
p.line(df['Time'], df['Price_Model1'], line_width=2)
show(p)


4. Model 2: Demand-Based Pricing

In [ ]:
# Dummy encoding vehicle types
vehicle_weights = {'car': 1.0, 'bike': 0.5, 'truck': 1.5}
df['VehicleWeight'] = df['VehicleType'].map(vehicle_weights)

# Demand function coefficients
α = 1.0
β = 0.8
γ = 0.5
δ = 2.0
ε = 1.0
λ = 0.3

def compute_demand(row):
    occ_rate = row['Occupancy'] / row['Capacity']
    return α * occ_rate + β * row['QueueLength'] - γ * row['Traffic'] + δ * row['IsSpecialDay'] + ε * row['VehicleWeight']

df['Demand'] = df.apply(compute_demand, axis=1)

# Normalize demand
df['NormDemand'] = (df['Demand'] - df['Demand'].min()) / (df['Demand'].max() - df['Demand'].min())

# Pricing function
df['Price_Model2'] = BASE_PRICE * (1 + λ * df['NormDemand'])

# Bound prices between 0.5x and 2x base
df['Price_Model2'] = df['Price_Model2'].clip(lower=BASE_PRICE * 0.5, upper=BASE_PRICE * 2)

df[['Time', 'Price_Model2']].head()


 5. Visualization (Model 2)

In [ ]:
p2 = figure(title="Model 2 - Demand-Based Price", x_axis_type='datetime', x_axis_label='Time', y_axis_label='Price')
p2.line(df['Time'], df['Price_Model2'], color='green', line_width=2)
show(p2)


6. Model 3 - Competitive Pricing Using Location

In [ ]:
from geopy.distance import geodesic

# Calculate distance matrix
def compute_distances(df):
    location_coords = df.groupby('ParkingLotID')[['Latitude', 'Longitude']].first()
    locations = location_coords.index.tolist()
    distance_matrix = pd.DataFrame(index=locations, columns=locations)

    for i in locations:
        for j in locations:
            if i != j:
                loc_i = location_coords.loc[i]
                loc_j = location_coords.loc[j]
                dist = geodesic((loc_i['Latitude'], loc_i['Longitude']), (loc_j['Latitude'], loc_j['Longitude'])).meters
                distance_matrix.loc[i, j] = dist
            else:
                distance_matrix.loc[i, j] = 0.0
    return distance_matrix.astype(float)

distance_matrix = compute_distances(df)
distance_matrix.head()


 7. Real-Time Simulation with Pathway

In [ ]:
# Simulate streaming using Pathway (basic structure)
class StreamingPricingModel(pw.Schema):
    time = pw.TimestampColumn()
    lot_id = pw.StringColumn()
    occupancy = pw.IntColumn()
    capacity = pw.IntColumn()
    queue_length = pw.IntColumn()
    traffic = pw.FloatColumn()
    is_special_day = pw.BoolColumn()
    vehicle_type = pw.StringColumn()
    price = pw.FloatColumn()

# This would be connected to a real-time source in full application
# Here, use batched feeding as simulation


8. Final Combined Visualization (Model 1 vs 2)

In [ ]:
p3 = figure(title="Comparison of Models", x_axis_type='datetime', x_axis_label='Time', y_axis_label='Price')
p3.line(df['Time'], df['Price_Model1'], legend_label="Model 1", line_width=2, color='blue')
p3.line(df['Time'], df['Price_Model2'], legend_label="Model 2", line_width=2, color='green')
p3.legend.location = "top_left"
show(p3)


In [ ]:
import pathway as pw

class DataSchema(pw.Schema):
    SystemCodeNumber: str
    Capacity: int
    Latitude: float
    Longitude: float
    Occupancy: int
    VehicleType: str
    TrafficConditionNearby: str
    QueueLength: int
    IsSpecialDay: int
    LastUpdatedDate: str
    LastUpdatedTime: str
    Timestamp: str

# Read the streamed data
t = pw.io.csv.read("streamed_data.csv", schema=DataSchema, mode="streaming", autocommit_duration_ms=200)

# Select a few columns without applying any UDF (keeping the simplified version for now)
result = t.select(
    t.Timestamp,
    t.SystemCodeNumber,
    t.Occupancy
)

# Write the result to a CSV file
pw.io.csv.write(result, "output.csv")

# Run the Pathway program
pw.run_local()